# Default title text
# RAG System - Asistente de Inversión en Dividendos

Este notebook implementa un sistema completo de **Retrieval-Augmented Generation (RAG)** especializado en inversión en dividendos.

## 🎯 Objetivo
Desarrollar un asistente inteligente capaz de responder preguntas específicas sobre estrategias de inversión en dividendos, análisis financiero y gestión de carteras, basándose en un extenso corpus de conocimiento especializado.

## 📚 Dataset de Conocimiento Financiero
- **93 documentos** de contenido financiero especializado
- **Pipeline de procesamiento avanzado**: Audio → Transcripción → Optimización con IA → Textos estructurados
- **Áreas de conocimiento**: Análisis fundamental, selección de brokers, psicología de la inversión, gestión de riesgo, fiscalidad, ratios financieros, etc.
- **Estructura modular**: Contenido organizado en 16 módulos temáticos + material complementario

## 🔧 Stack Tecnológico
- **LangChain**: Framework para aplicaciones RAG
- **OpenAI**: Embeddings (text-embedding-3-large) y LLM (GPT-4o-mini)
- **FAISS**: Base de datos vectorial para búsqueda semántica eficiente
- **Python**: Procesamiento de datos y análisis

## 📦 Instalación de Dependencias

In [ ]:
!pip install langchain langchain-openai langchain-community faiss-cpu tiktoken tqdm pandas matplotlib seaborn openai


## 💾 Configuración de Google Drive y Verificación de Chunks

Montamos Google Drive para persistir los chunks procesados y evitar repetir el proceso de extracción de metadatos con GPT-4o.

In [ ]:
from google.colab import drive
import pickle
import os

print("📁 Montando Google Drive...")
drive.mount('/content/drive')
print("✅ Google Drive montado correctamente")

DRIVE_FOLDER = '/content/drive/MyDrive/RAG_Dividendos/'
CHUNKS_PATH = '/content/drive/MyDrive/RAG_Dividendos/chunks_procesados.pkl'

os.makedirs(DRIVE_FOLDER, exist_ok=True)
print(f"📂 Carpeta de trabajo: {DRIVE_FOLDER}")


🔍 Verificar si ya existen chunks procesados

In [ ]:
print("🔍 Verificando chunks existentes en Google Drive...")

chunks_exist = os.path.exists(CHUNKS_PATH)

if chunks_exist:
    print("✅ ¡Chunks encontrados en Google Drive!")
    print(f"📁 Ruta: {CHUNKS_PATH}")

    try:
        print("📥 Cargando chunks desde Google Drive...")
        with open(CHUNKS_PATH, 'rb') as f:
            chunks = pickle.load(f)

        print(f"🎉 ¡Chunks cargados exitosamente!")
        print(f"   📊 Total de chunks: {len(chunks)}")
        print(f"   📚 Documentos únicos: {len(set([chunk.metadata['source_file'] for chunk in chunks]))}")

        if chunks:
            sample_chunk = chunks[0]
            has_metadata = 'main_topic' in sample_chunk.metadata and 'level' in sample_chunk.metadata

            if has_metadata:
                print(f"✅ Metadatos verificados - chunks listos para usar")
                print(f"\n📋 Ejemplo de chunk cargado:")
                print(f"   🆔 Chunk ID: {sample_chunk.metadata.get('chunk_id', 'N/A')}")
                print(f"   📁 Archivo: {sample_chunk.metadata.get('filename', 'N/A')}")
                print(f"   🎯 Tema: {sample_chunk.metadata.get('main_topic', 'N/A')}")
                print(f"   📊 Nivel: {sample_chunk.metadata.get('level', 'N/A')}")
                print(f"   🔤 Tokens: {sample_chunk.metadata.get('chunk_tokens', 'N/A')}")

                print(f"\n⚡ SALTANDO TODO EL PROCESAMIENTO - chunks listos para RAG")
                SKIP_PROCESSING = True
            else:
                print(f"⚠️ Chunks encontrados pero sin metadatos completos")
                print(f"🔄 Se ejecutará el procesamiento completo")
                SKIP_PROCESSING = False
        else:
            print(f"❌ Archivo de chunks vacío")
            SKIP_PROCESSING = False

    except Exception as e:
        print(f"❌ Error cargando chunks: {e}")
        print(f"🔄 Se ejecutará el procesamiento completo")
        SKIP_PROCESSING = False
        chunks = None

else:
    print("❌ No se encontraron chunks en Google Drive")
    print(f"🔄 Se ejecutará el procesamiento completo desde cero")
    SKIP_PROCESSING = False
    chunks = None

print(f"\n{'='*50}")
if SKIP_PROCESSING:
    print("🚀 MODO RÁPIDO: Usando chunks existentes")
    print("💡 Puedes saltar directamente a la implementación del RAG")
else:
    print("🔄 MODO PROCESAMIENTO: Generando chunks desde cero")
    print("💡 Ejecuta las siguientes celdas para procesar el dataset")
print(f"{'='*50}")


## 🔑 Configuración de API Keys

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Introduce tu OpenAI API Key: ")


## 📊 Análisis del Dataset de Conocimiento Financiero

En esta sección analizaremos la estructura y características del dataset procesado, que contiene conocimiento especializado sobre inversión en dividendos.

### 🎵 Pipeline de Procesamiento del Dataset

Antes de proceder con el análisis, es importante entender cómo se ha generado este dataset de conocimiento financiero:

#### **1. 🎙️ Extracción de Audio a Texto**
- **Fuente original**: Contenido de audio especializado en inversión en dividendos
- **Herramienta utilizada**: OpenAI Whisper para transcripción automática
- **Resultado**: Transcripciones en bruto de los audios

#### **2. ✂️ Segmentación Inicial**
- **Objetivo**: Dividir el contenido en fragmentos manejables
- **Límite**: Respetar los límites de tokens de la API de OpenAI
- **Método**: Chunking preservando contexto (4.000 tokens - 300 overlap tokens) - (GPT-4o ~4.096 tokens máximos en la respuesta)

#### **3. 🧠 Limpieza y Optimización con IA**
- **Herramienta**: GPT-4o
- **Proceso**: Refinamiento del contenido transcrito para mejorar claridad y estructura
- **Prompt utilizado**:


Eres un redactor profesional especializado en convertir transcripciones de voz en textos escritos claros, concisos y de alta calidad para publicaciones financieras.

Tienes como entrada la transcripción de una presentación sobre inversión en dividendos. Tu tarea es reescribir el texto para que:

ELIMINAR COMPLETAMENTE:
- Muletillas, repeticiones, frases interrumpidas y expresiones orales (como "ehh", "vale", "como decíamos", "bueno", "entonces").
- Referencias a cursos, módulos, lecciones, clases, alumnos, estudiantes o cualquier contexto educativo.
- Instrucciones técnicas de plataformas (como "haz clic", "descarga", "deja tu comentario").
- Expresiones dirigidas directamente al lector ("tú", "vosotros", "ustedes").
- Markdown innecesario, emojis decorativos, saltos de línea excesivos (`<br>`).

TRANSFORMAR EN:
- Un texto profesional de divulgación financiera, como si fuera parte de un libro especializado o artículo de revista económica.
- Redacción en tercera persona o impersonal, con tono objetivo y profesional.
- Estructura clara con párrafos bien organizados.
- Terminología técnica precisa sin simplificaciones excesivas.

MANTENER:
- Toda la información técnica y financiera relevante sobre dividendos.
- Datos, cifras, ejemplos prácticos y estrategias de inversión.
- Conceptos, definiciones y explicaciones técnicas.

MANEJO DE SOLAPAMIENTO:
Entre las etiquetas `{overlap_start}` y `{overlap_end}` encontrarás texto que ya apareció en el bloque anterior:
- No lo reescribas a menos que sea necesario para la continuidad.
- Evita repetir ideas ya tratadas.

Si encuentras alguna palabra, frase o fragmento que no entiendas o no puedas mejorar o contenido que no puedas procesar adecuadamente, déjalo tal cual pero márcalo entre etiquetas `<unprocessed>...texto...</unprocessed>` para que pueda revisarse manualmente más adelante.

No inventes información nueva. Reescribe el texto original, pero mejora la redacción y elimina el contenido innecesario.
El resultado debe ser un texto que parezca extraído de una publicación financiera profesional, sin rastro de su origen como material educativo.


#### **4. 📁 Estructuración Final**
- **Organización**: Un archivo .txt por lección
- **Formato**: Texto plano listo para chunking con LangChain

### 🔄 Pipeline RAG en este Notebook

En este notebook completaremos el pipeline RAG:

1. **📚 Document Loading**: Cargar archivos .txt con LangChain
2. **✂️ Chunking**: Segmentación con RecursiveCharacterTextSplitter
3. **🧠 Metadata Extraction**: Usar GPT-4o para extraer `level` y `main_topic` por documento
4. **🗄️ Vector Store**: Crear índice semántico con FAISS y embeddings OpenAI
5. **🔍 Retrieval**: Sistema de búsqueda semántica con filtros por metadatos
6. **🤖 RAG Pipeline**: Cadena completa de pregunta-respuesta con GPT-4o-mini

### 📂 Carga del Dataset Procesado

Para ejecutar este notebook, necesitas subir los archivos de texto procesados.

.

### 📋 Instrucciones para subir el dataset

**📦 Pasos para cargar los archivos de texto procesados:**
1. 📁 Comprime los archivos .txt procesados en un archivo llamado `data.zip`
2. 📤 Haz clic en el icono de carpeta 📁 en la barra lateral izquierda
3. 🔄 Arrastra y suelta `data.zip` o usa "Subir archivos"
4. ⏳ Espera a que se complete la subida
5. ▶️ Ejecuta las celdas siguientes para:
   - Extraer y cargar documentos con LangChain
   - Aplicar chunking
   - Extraer metadatos con GPT-4o (`level` y `main_topic`)

In [ ]:
import json
import glob
import os
import zipfile


📤 Extraer y cargar archivos del dataset

In [ ]:
if not SKIP_PROCESSING:
    print("🔍 Buscando archivos del dataset...")

    if os.path.exists('data.zip'):
        print("✅ Archivo data.zip encontrado. Extrayendo...")
        with zipfile.ZipFile('data.zip', 'r') as zip_ref:
            zip_ref.extractall('.')
        print("✅ Archivos extraídos correctamente")
    else:
        print("❌ No se encontró data.zip")
        print("\n📋 PASOS A SEGUIR:")
        print("1. Ve al panel de archivos (📁) en la barra lateral izquierda")
        print("2. Haz clic en 'Subir archivos' o arrastra data.zip")
        print("3. Espera a que se complete la subida")
        print("4. Vuelve a ejecutar esta celda")

    txt_files = glob.glob("data/*.txt")
    print(f"\n📁 Archivos de texto encontrados en 'data/': {len(txt_files)}")

    if len(txt_files) == 0:
        print("❌ No se encontraron archivos de texto en la carpeta 'data'")
        print("🔄 Asegúrate de haber subido data.zip correctamente y que contenga la carpeta 'data' con archivos .txt")
    else:
        print(f"✅ Dataset encontrado: {len(txt_files)} lecciones de conocimiento financiero")
else:
    print("⚡ SALTANDO: Ya tenemos chunks procesados desde Google Drive")
    txt_files = []

## 📚 Document Loading con LangChain

En esta sección cargaremos los documentos de texto utilizando LangChain y extraeremos metadatos desde los nombres de archivo.

In [ ]:
if not SKIP_PROCESSING:
    import re
    from langchain_community.document_loaders import DirectoryLoader, TextLoader
    from langchain.schema import Document
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    import tiktoken

    def extract_metadata_from_filename(filename):
        base_name = filename.replace('.txt', '')
        parts = base_name.split('_')

        if len(parts) >= 2:
            try:
                module = int(parts[0])
                lesson = int(parts[1])

                if 'bonus' in base_name.lower():
                    content_type = "bonus"
                elif 'conclusion' in base_name.lower():
                    content_type = "conclusión"
                elif lesson == 0:
                    content_type = "introducción"
                else:
                    content_type = "lección"

                return {
                    'source_file': base_name,
                    'module': module,
                    'lesson': lesson,
                    'content_type': content_type,
                    'filename': filename
                }
            except (ValueError, IndexError):
                pass

        return {
            'source_file': base_name,
            'module': 0,
            'lesson': 0,
            'content_type': 'contenido',
            'filename': filename
        }

    print("🔧 Funciones de extracción de metadatos configuradas")
else:
    print("⚡ SALTANDO: Funciones de procesamiento no necesarias")


📖 Cargar documentos con LangChain

In [ ]:
if not SKIP_PROCESSING:
    print("📚 Cargando documentos con LangChain...")

    if len(txt_files) > 0:
        raw_documents = []
        for txt_file in txt_files:
            try:
                loader = TextLoader(txt_file, encoding='utf-8')
                doc = loader.load()[0]
                raw_documents.append(doc)
            except Exception as e:
                print(f"⚠️ Error cargando {txt_file}: {e}")

        print(f"✅ Cargados {len(raw_documents)} documentos base")

        enriched_documents = []
        for doc in raw_documents:
            filename = os.path.basename(doc.metadata['source'])
            basic_metadata = extract_metadata_from_filename(filename)
            doc.metadata.update(basic_metadata)
            enriched_documents.append(doc)

        print(f"✅ Documentos enriquecidos con metadatos básicos")

        if enriched_documents:
            sample_doc = enriched_documents[0]
            print(f"\n📋 Ejemplo de metadatos básicos:")
            print(f"📁 Archivo: {sample_doc.metadata['filename']}")
            print(f"📚 Módulo: {sample_doc.metadata['module']}")
            print(f"📖 Lección: {sample_doc.metadata['lesson']}")
            print(f"🔖 Tipo: {sample_doc.metadata['content_type']}")
            content_preview = sample_doc.page_content[:200] + "..." if len(sample_doc.page_content) > 200 else sample_doc.page_content
            print(f"\n📝 Vista previa del contenido:")
            print(f"'{content_preview}'")

            print(f"\n💡 Nota: Los metadatos 'level' y 'main_topic' se extraerán con GPT-4o automáticamente")
    else:
        print("❌ No hay documentos para cargar")
else:
    print("⚡ SALTANDO: Carga de documentos no necesaria")


## ✂️ Chunking Strategy con LangChain

Implementaremos una estrategia de chunking utilizando el `RecursiveCharacterTextSplitter` de LangChain, optimizado para contenido financiero.

✂️ Configurar Text Splitter optimizado para contenido financiero

In [ ]:
if not SKIP_PROCESSING:
    print("🔧 Configurando estrategia de chunking...")

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=150,
        length_function=len,
        separators=["\n\n", "\n", ". "],
        add_start_index=True
    )

    print("✅ Text Splitter configurado:")
    print(f"   📏 Chunk size: {text_splitter._chunk_size}")
    print(f"   🔗 Overlap: {text_splitter._chunk_overlap}")
    print(f"   📊 Separadores: {len(text_splitter._separators)} niveles")
else:
    print("⚡ SALTANDO: Configuración de chunking no necesaria")


🔄 Aplicar chunking a los documentos

In [ ]:
if not SKIP_PROCESSING:
    print("✂️ Aplicando chunking a los documentos...")

    if 'enriched_documents' in locals() and enriched_documents:
        chunks = text_splitter.split_documents(enriched_documents)

        for i, chunk in enumerate(chunks):
            chunk.metadata['chunk_id'] = i
            chunk.metadata['chunk_length'] = len(chunk.page_content)

            try:
                encoding = tiktoken.encoding_for_model("gpt-4o")
                chunk.metadata['chunk_tokens'] = len(encoding.encode(chunk.page_content))
            except:
                chunk.metadata['chunk_tokens'] = len(chunk.page_content) // 4

        print(f"✅ Chunking completado:")
        print(f"   📚 Documentos originales: {len(enriched_documents)}")
        print(f"   🧩 Chunks generados: {len(chunks)}")
        print(f"   📊 Promedio chunks por documento: {len(chunks) / len(enriched_documents):.1f}")

        chunk_lengths = [chunk.metadata['chunk_length'] for chunk in chunks]
        chunk_tokens = [chunk.metadata['chunk_tokens'] for chunk in chunks]

        print(f"\n📈 Estadísticas de chunks:")
        print(f"   📏 Longitud promedio: {sum(chunk_lengths) / len(chunk_lengths):.0f} caracteres")
        print(f"   🔤 Tokens promedio: {sum(chunk_tokens) / len(chunk_tokens):.0f} tokens")
        print(f"   📊 Longitud mín/máx: {min(chunk_lengths)}/{max(chunk_lengths)} caracteres")
        print(f"   🎯 Tokens mín/máx: {min(chunk_tokens)}/{max(chunk_tokens)} tokens")

        if chunks:
            sample_chunk = chunks[0]
            print(sample_chunk)
            print(f"\n📋 Ejemplo de chunk generado:")
            print(f"   🆔 Chunk ID: {sample_chunk.metadata['chunk_id']}")
            print(f"   📁 Documento origen: {sample_chunk.metadata['filename']}")
            print(f"   📏 Longitud: {sample_chunk.metadata['chunk_length']} caracteres")
            print(f"   🔤 Tokens: {sample_chunk.metadata['chunk_tokens']} tokens")
            preview = sample_chunk.page_content[:300] + "..." if len(sample_chunk.page_content) > 300 else sample_chunk.page_content
            print(f"   📝 Contenido: '{preview}'")

    else:
        print("❌ No hay documentos cargados para hacer chunking")
else:
    print("⚡ SALTANDO: Chunking no necesario")


# 🤖 Extracción de Metadatos con GPT-4o

Esta sección utiliza **GPT-4o** para analizar automáticamente el contenido de cada documento y extraer metadatos clave:

- **📋 main_topic**: Identifica el tema principal de cada lección (ej: "Ventajas de invertir en dividendos")
- **📊 level**: Determina el nivel de dificultad del contenido (básico, intermedio, avanzado)

Estos metadatos enriquecen los chunks y permiten filtros más precisos durante la recuperación de información en el sistema RAG.

In [ ]:
if not SKIP_PROCESSING:
    from openai import OpenAI
    import tiktoken
    import json as json_lib
    from collections import defaultdict

    client = OpenAI()

    encoding = tiktoken.encoding_for_model("gpt-4o")

    def count_tokens(text):
        return len(encoding.encode(text))

    def truncate_text_if_needed(text, max_tokens=8000):
        tokens = encoding.encode(text)
        if len(tokens) <= max_tokens:
            return text, len(tokens)

        truncated_tokens = tokens[:max_tokens]
        truncated_text = encoding.decode(truncated_tokens)
        print(f"⚠️  Texto truncado de {len(tokens)} a {len(truncated_tokens)} tokens")
        return truncated_text, len(truncated_tokens)

    def analyze_content_with_gpt4o(text):
        final_text, num_tokens = truncate_text_if_needed(text)

        prompt = f"""
        Eres un asistente experto en educación financiera. Se te da el contenido de una lección sobre inversión en dividendos.

        Tu tarea es:
        1. Identificar el tema principal (una frase corta, como "Ventajas de invertir en dividendos").
        2. Estimar el nivel de dificultad del contenido (básico, intermedio o avanzado).

        Contenido de la lección ({num_tokens} tokens):
        \"\"\"\n{final_text}\n\"\"\"

        RESPONDE ÚNICAMENTE CON EL JSON, SIN MARKDOWN, SIN EXPLICACIONES, SIN TEXTO ADICIONAL:

        {{
            "main_topic": "...",
            "level": "básico | intermedio | avanzado"
        }}
        """

        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = client.chat.completions.create(
                    model="gpt-4o",
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.1,
                    max_tokens=150
                )

                response_text = response.choices[0].message.content.strip()
                
                try:
                    return json_lib.loads(response_text)
                except json_lib.JSONDecodeError:
                    import re
                    json_match = re.search(r'\{[^}]+\}', response_text)
                    if json_match:
                        try:
                            return json_lib.loads(json_match.group())
                        except:
                            pass

                    if attempt < max_retries - 1:
                        continue
                    else:
                        raise

            except Exception as e:
                if attempt < max_retries - 1:
                    continue
                else:
                    raise e

        return {"main_topic": "Contenido financiero", "level": "intermedio"}

    print("🔧 Funciones de extracción de metadatos con GPT-4o configuradas")
else:
    print("⚡ SALTANDO: Funciones de GPT-4o no necesarias")


🧠 Extraer metadatos por documento

In [ ]:
if not SKIP_PROCESSING:
    print("🤖 Extrayendo metadatos con GPT-4o...")

    if 'chunks' in locals() and chunks:
        docs_by_source = defaultdict(list)
        for chunk in chunks:
            source_file = chunk.metadata['source_file']
            docs_by_source[source_file].append(chunk)

        print(f"📚 Procesando {len(docs_by_source)} documentos únicos...")

        metadata_cache = {}

        for i, (source_file, doc_chunks) in enumerate(docs_by_source.items()):
            print(f"\n🔄 Procesando ({i+1}/{len(docs_by_source)}): {source_file}")
            full_content = " ".join([chunk.page_content for chunk in doc_chunks])

            try:
                metadata = analyze_content_with_gpt4o(full_content)
                metadata_cache[source_file] = metadata

                print(f"   ✅ Tema: {metadata['main_topic']}")
                print(f"   📊 Nivel: {metadata['level']}")

            except Exception as e:
                print(f"   ❌ Error: {e}")
                metadata_cache[source_file] = {
                    "main_topic": "Contenido financiero",
                    "level": "intermedio"
                }

        print(f"\n🔄 Aplicando metadatos a {len(chunks)} chunks...")

        for chunk in chunks:
            source_file = chunk.metadata['source_file']
            if source_file in metadata_cache:
                chunk.metadata['main_topic'] = metadata_cache[source_file]['main_topic']
                chunk.metadata['level'] = metadata_cache[source_file]['level']

        print(f"✅ Metadatos aplicados a todos los chunks")

        if chunks:
            sample_chunk = chunks[0]
            print(f"\n📋 Ejemplo de chunk con metadatos completos:")
            print(f"   🆔 Chunk ID: {sample_chunk.metadata['chunk_id']}")
            print(f"   📁 Archivo: {sample_chunk.metadata['filename']}")
            print(f"   📚 Módulo: {sample_chunk.metadata['module']}")
            print(f"   📖 Lección: {sample_chunk.metadata['lesson']}")
            print(f"   🎯 Tema: {sample_chunk.metadata['main_topic']}")
            print(f"   📊 Nivel: {sample_chunk.metadata['level']}")
            print(f"   🔖 Tipo: {sample_chunk.metadata['content_type']}")
            print(f"   🔤 Tokens: {sample_chunk.metadata['chunk_tokens']}")

        print(f"\n💾 Guardando chunks procesados en Google Drive...")
        try:
            with open(CHUNKS_PATH, 'wb') as f:
                pickle.dump(chunks, f)
            print(f"✅ {len(chunks)} chunks guardados exitosamente en Google Drive")
            print(f"📁 Ruta: {CHUNKS_PATH}")
            print(f"💡 En futuras ejecuciones se cargarán automáticamente desde aquí")
        except Exception as e:
            print(f"❌ Error guardando chunks: {e}")

    else:
        print("❌ No hay chunks disponibles para extraer metadatos")
else:
    print("⚡ SALTANDO: Extracción de metadatos no necesaria")

print("✅ ¡Dataset listo para implementar el sistema RAG completo!")

## 🗄️ Vector Store con FAISS

Implementaremos el vector store usando FAISS para búsqueda semántica sobre nuestros chunks procesados con metadatos completos.


💾 Verificación de Vector Store existente en Google Drive

In [ ]:
print("🔍 Verificando vector store existente en Google Drive...")

VECTORSTORE_PATH = '/content/drive/MyDrive/RAG_Dividendos/vectorstore_faiss'
VECTORSTORE_INDEX_PATH = '/content/drive/MyDrive/RAG_Dividendos/vectorstore_faiss/index.faiss'
VECTORSTORE_PKL_PATH = '/content/drive/MyDrive/RAG_Dividendos/vectorstore_faiss/index.pkl'

vectorstore_exists = os.path.exists(VECTORSTORE_INDEX_PATH) and os.path.exists(VECTORSTORE_PKL_PATH)

if vectorstore_exists:
    print("✅ ¡Vector store encontrado en Google Drive!")
    print(f"📁 Índice FAISS: {VECTORSTORE_INDEX_PATH}")
    print(f"📁 Metadatos: {VECTORSTORE_PKL_PATH}")
    
    try:
        from langchain_community.vectorstores import FAISS
        from langchain_openai.embeddings import OpenAIEmbeddings
        
        embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
        
        print("📥 Cargando vector store desde Google Drive...")
        vectorstore = FAISS.load_local(
            VECTORSTORE_PATH, 
            embeddings,
            allow_dangerous_deserialization=True
        )
        
        print(f"🎉 ¡Vector store cargado exitosamente!")
        print(f"   📊 Vectores en el índice: {vectorstore.index.ntotal}")
        print(f"   📐 Dimensiones: {vectorstore.index.d}")
        
        print(f"\n⚡ SALTANDO CREACIÓN DE VECTOR STORE - usando existente")
        SKIP_VECTORSTORE = True
        
    except Exception as e:
        print(f"❌ Error cargando vector store: {e}")
        print(f"🔄 Se creará un nuevo vector store")
        SKIP_VECTORSTORE = False
        vectorstore = None
        
else:
    print("❌ No se encontró vector store en Google Drive")
    print(f"🔄 Se creará un nuevo vector store desde cero")
    SKIP_VECTORSTORE = False
    vectorstore = None

print(f"\n{'='*50}")
if SKIP_VECTORSTORE:
    print("🚀 MODO RÁPIDO: Usando vector store existente")
    print("💡 Listo para búsquedas semánticas")
else:
    print("🔄 MODO CREACIÓN: Generando vector store desde cero")
    print("💡 Se crearán embeddings para todos los chunks")
print(f"{'='*50}")


🧠 Creación del Vector Store con FAISS

In [ ]:
if not SKIP_VECTORSTORE:
    print("🔧 Creando vector store con FAISS...")
    
    if 'chunks' in locals() and chunks and len(chunks) > 0:
        print(f"📊 Procesando {len(chunks)} chunks para crear embeddings...")
        
        from langchain_community.vectorstores import FAISS
        from langchain_openai.embeddings import OpenAIEmbeddings
        import time
        
        print("🔧 Inicializando OpenAI Embeddings (text-embedding-3-large)...")
        embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
        
        print("⏳ Creando embeddings y construyendo índice FAISS...")
        print("💡 Este proceso puede tomar varios minutos dependiendo del número de chunks...")
        
        start_time = time.time()
        
        try:
            vectorstore = FAISS.from_documents(chunks, embedding=embeddings)
            
            end_time = time.time()
            processing_time = end_time - start_time
            
            print(f"✅ Vector store creado exitosamente!")
            print(f"   ⏱️ Tiempo de procesamiento: {processing_time:.1f} segundos")
            print(f"   📊 Vectores en el índice: {vectorstore.index.ntotal}")
            print(f"   📐 Dimensiones del embedding: {vectorstore.index.d}")
            print(f"   🧩 Chunks procesados: {len(chunks)}")
            
            if chunks:
                sample_doc = vectorstore.similarity_search("dividendos", k=1)[0]
                print(f"\n📋 Verificación de metadatos en vector store:")
                print(f"   📁 Archivo: {sample_doc.metadata.get('filename', 'N/A')}")
                print(f"   🎯 Tema: {sample_doc.metadata.get('main_topic', 'N/A')}")
                print(f"   📊 Nivel: {sample_doc.metadata.get('level', 'N/A')}")
                print(f"   📚 Módulo: {sample_doc.metadata.get('module', 'N/A')}")
                
        except Exception as e:
            print(f"❌ Error creando vector store: {e}")
            print("🔄 Verifica tu API key de OpenAI y conexión a internet")
            vectorstore = None
            
    else:
        print("❌ No hay chunks disponibles para crear el vector store")
        print("🔄 Asegúrate de haber ejecutado las celdas de procesamiento de chunks")
        vectorstore = None
        
else:
    print("⚡ SALTANDO: Vector store ya cargado desde Google Drive")


💾 Persistencia del Vector Store en Google Drive

In [ ]:
if not SKIP_VECTORSTORE and vectorstore is not None:
    print("💾 Guardando vector store en Google Drive...")
    
    try:
        vectorstore.save_local(VECTORSTORE_PATH)
        
        print(f"✅ Vector store guardado exitosamente en Google Drive!")
        print(f"📁 Ruta base: {VECTORSTORE_PATH}")
        print(f"📁 Índice FAISS: {VECTORSTORE_INDEX_PATH}")
        print(f"📁 Metadatos: {VECTORSTORE_PKL_PATH}")
        print(f"💡 En futuras ejecuciones se cargará automáticamente desde aquí")
        
        if os.path.exists(VECTORSTORE_INDEX_PATH) and os.path.exists(VECTORSTORE_PKL_PATH):
            index_size = os.path.getsize(VECTORSTORE_INDEX_PATH) / (1024*1024)
            pkl_size = os.path.getsize(VECTORSTORE_PKL_PATH) / (1024*1024)
            print(f"📊 Tamaño del índice: {index_size:.1f} MB")
            print(f"📊 Tamaño de metadatos: {pkl_size:.1f} MB")
        else:
            print("⚠️ Advertencia: No se pudieron verificar todos los archivos guardados")
            
    except Exception as e:
        print(f"❌ Error guardando vector store: {e}")
        print("⚠️ El vector store se ha creado pero no se pudo guardar en Google Drive")
        print("💡 Podrás usarlo en esta sesión, pero se perderá al reiniciar")
        
elif SKIP_VECTORSTORE:
    print("⚡ SALTANDO: Vector store ya existía en Google Drive")
else:
    print("❌ No hay vector store para guardar (no se creó correctamente)")


🔍 Configuración del Retriever para búsquedas semánticas

In [ ]:
print("🔧 Configurando retriever para búsquedas semánticas...")

if vectorstore is not None:
    # 1. Retriever básico por similitud
    basic_retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5} 
    )
    
    # 2. Retriever con threshold de similitud (más estricto)
    similarity_retriever = vectorstore.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={"k": 10, "score_threshold": 0.5}
    )
    
    # 3. Retriever con MMR (Maximum Marginal Relevance) para diversidad
    mmr_retriever = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={
            "k": 5,
            "fetch_k": 20,              # Buscar entre los top 20
            "lambda_mult": 0.7          # Balance relevancia vs diversidad
        }
    )
    
    print("✅ Retrievers configurados:")
    print("   🔍 basic_retriever: Top 5 chunks más similares")
    print("   🎯 similarity_retriever: Chunks con similitud > 0.7")
    print("   🌈 mmr_retriever: Top 5 con diversidad (MMR)")
    
    def search_with_filters(query, level=None, module=None, content_type=None, k=5):
        filter_dict = {}
        
        if level:
            filter_dict['level'] = level
        if module:
            filter_dict['module'] = module
        if content_type:
            filter_dict['content_type'] = content_type
        
        if filter_dict:
            results = vectorstore.similarity_search_with_score(query, k=k, filter=filter_dict)
        else:
            results = vectorstore.similarity_search_with_score(query, k=k)
        
        return results
    
    print("✅ Función de búsqueda con filtros configurada")
    print("💡 Usa search_with_filters(query, level='intermedio', module=2, k=3)")
    
else:
    print("❌ No se puede configurar retriever: vector store no disponible")
    print("🔄 Asegúrate de haber creado o cargado el vector store correctamente")


### 🧪 Pruebas del Vector Store con consultas sobre inversión en dividendos

In [ ]:
print("🧪 Probando el vector store con consultas de ejemplo...")

if vectorstore is not None:
    test_queries = [
        "¿Qué son los dividendos y por qué las empresas los pagan?",
        "¿Cuáles son las ventajas de invertir en dividendos?",
        "¿Cómo seleccionar acciones que paguen dividendos?",
        "¿Qué es la rentabilidad por dividendo?",
        "¿Cuáles son los riesgos de la inversión en dividendos?"
    ]
    
    print(f"📋 Ejecutando {len(test_queries)} consultas de prueba...\n")
    
    for i, query in enumerate(test_queries, 1):
        print(f"🔍 Consulta {i}: '{query}'")
        
        try:
            results = vectorstore.similarity_search_with_score(query, k=3)
            
            print(f"   📊 Encontrados {len(results)} chunks relevantes:")
            
            for j, (doc, score) in enumerate(results, 1):
                print(f"   {j}. 📄 {doc.metadata.get('filename', 'N/A')[:50]}...")
                print(f"      🎯 Tema: {doc.metadata.get('main_topic', 'N/A')}")
                print(f"      📊 Nivel: {doc.metadata.get('level', 'N/A')}")
                print(f"      🔢 Score: {score:.3f}")
                print(f"      📝 Contenido: {doc.page_content[:100]}...")
                print()
                
        except Exception as e:
            print(f"   ❌ Error en búsqueda: {e}")
            
        print("-" * 80)
    
    print("\n🎯 Prueba con filtros por metadatos:")
    print("🔍 Búsqueda: 'dividendos' solo en contenido intermedio...")
    
    try:
        filtered_results = search_with_filters("dividendos", level="intermedio", k=3)
        
        print(f"   📊 Encontrados {len(filtered_results)} chunks intermedios:")
        for i, (doc, score) in enumerate(filtered_results, 1):
            print(f"   {i}. 📄 {doc.metadata.get('filename', 'N/A')}")
            print(f"      📊 Nivel: {doc.metadata.get('level', 'N/A')}")
            print(f"      📚 Módulo: {doc.metadata.get('module', 'N/A')}")
            print(f"      🔢 Score: {score:.3f}")
            
    except Exception as e:
        print(f"   ❌ Error en búsqueda filtrada: {e}")
    
    print("\n✅ ¡Vector store funcionando correctamente!")
    print("💡 Listo para implementar el pipeline RAG completo")
    
else:
    print("❌ No se pueden ejecutar pruebas: vector store no disponible")
    print("🔄 Asegúrate de haber creado o cargado el vector store correctamente")


## 🤖 Pipeline RAG Completo

Implementaremos el sistema completo de pregunta-respuesta combinando retrieval semántico con generación de respuestas usando GPT-4o-mini.


🔧 Configuración del LLM y Prompt Template especializado

In [ ]:
print("🔧 Configurando LLM y prompt template para finanzas...")

if vectorstore is not None:
    from langchain_openai import ChatOpenAI
    from langchain.prompts import PromptTemplate
    from langchain_core.output_parsers import StrOutputParser
    from langchain_core.runnables import RunnablePassthrough
    
    base_model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1, max_tokens=1000)
    
    financial_prompt_template = """
    Eres un asistente experto en inversión en dividendos y análisis financiero. Tu objetivo es proporcionar respuestas precisas, educativas y prácticas basándose únicamente en el contexto proporcionado.

    INSTRUCCIONES:
    1. Responde ÚNICAMENTE basándote en el contexto proporcionado
    2. Si la información no está en el contexto, responde de forma BREVE usando una de estas variaciones:
        - "No tengo información sobre [tema] en mi base de conocimiento financiero."
        - "Esta consulta está fuera de mi especialización en dividendos."
        - "No encuentro información sobre [tema] en el contexto proporcionado."
        - "Mi conocimiento se limita a inversión en dividendos."
        - "No puedo ayudar con [tema], solo con temas financieros."
    3. Usa un lenguaje claro y profesional apropiado para inversores
    4. Incluye ejemplos prácticos cuando sea relevante
    5. Estructura tu respuesta de forma clara y organizada
    6. Si mencionas conceptos técnicos, explícalos brevemente
    
    CONTEXTO RELEVANTE:
    {context}

    PREGUNTA DEL USUARIO:
    {question}

    RESPUESTA EXPERTA:
    """

    financial_prompt = PromptTemplate(template=financial_prompt_template, input_variables=["context", "question"])
    
    print("✅ LLM configurado:")
    print(f"   🤖 Modelo: {base_model.model_name}")
    print(f"   🌡️ Temperature: {base_model.temperature}")
    print(f"   📝 Max tokens: {base_model.max_tokens}")
    
    print("✅ Prompt template especializado en finanzas creado")
    print("💡 Optimizado para respuestas educativas y precisas sobre inversión en dividendos")
    
else:
    print("❌ No se puede configurar el pipeline RAG: vector store no disponible")
    print("🔄 Asegúrate de haber creado o cargado el vector store correctamente")


🔗 Creación de la cadena RAG (Retrieval + Generation)

In [ ]:
print("🔗 Creando cadena RAG completa...")

if vectorstore is not None and 'base_model' in locals():
    rag_chain = (
        {"context": basic_retriever, "question": RunnablePassthrough()}
        | financial_prompt
        | base_model
        | StrOutputParser()
    )
    
    rag_chain_similarity = (
        {"context": similarity_retriever, "question": RunnablePassthrough()}
        | financial_prompt
        | base_model
        | StrOutputParser()
    )
    
    rag_chain_mmr = (
        {"context": mmr_retriever, "question": RunnablePassthrough()}
        | financial_prompt
        | base_model
        | StrOutputParser()
    )
    
    print("✅ Cadenas RAG creadas exitosamente:")
    print("   🔗 rag_chain: Cadena principal (retriever básico)")
    print("   🎯 rag_chain_similarity: Con threshold de similitud")
    print("   🌈 rag_chain_mmr: Con diversidad (MMR)")
    
    print("\n💡 Características de las cadenas:")
    print("   🚀 Método: LCEL (LangChain Expression Language)")
    print("   ⚡ Más rápido y eficiente que RetrievalQA")
    print("   🎯 Prompt: Especializado en finanzas")
    
else:
    print("❌ No se pueden crear las cadenas RAG")
    if vectorstore is None:
        print("   - Vector store no disponible")
    if 'base_model' not in locals():
        print("   - LLM no configurado")
    print("🔄 Asegúrate de haber ejecutado las celdas anteriores correctamente")


🎯 Funciones helper para consultas especializadas

In [ ]:
print("🎯 Creando funciones helper para consultas...")

if 'rag_chain' in locals():
    
    def ask_dividends_expert(question, retriever_type="basic", show_sources=True):
        if retriever_type == "similarity":
            chain = rag_chain_similarity
            retriever = similarity_retriever
        elif retriever_type == "mmr":
            chain = rag_chain_mmr
            retriever = mmr_retriever
        else:
            chain = rag_chain
            retriever = basic_retriever
        
        try:
            answer = chain.invoke(question)

            sources = []
            if show_sources:
                sources = retriever.invoke(question)
            
            response = {
                "question": question,
                "answer": answer,
                "sources": sources,
                "retriever_type": retriever_type
            }
            
            if show_sources:
                print(f"🤖 **RESPUESTA DEL EXPERTO:**")
                print(f"{answer}")
                print(f"\n📚 **FUENTES CONSULTADAS ({len(response['sources'])} documentos):**")
                
                for i, doc in enumerate(response['sources'], 1):
                    print(f"\n{i}. 📄 **{doc.metadata.get('filename', 'N/A')}**")
                    print(f"   🎯 Tema: {doc.metadata.get('main_topic', 'N/A')}")
                    print(f"   📊 Nivel: {doc.metadata.get('level', 'N/A')}")
                    print(f"   📚 Módulo: {doc.metadata.get('module', 'N/A')}")
                    print(f"   📝 Extracto: {doc.page_content[:150]}...")
            
            return response
            
        except Exception as e:
            print(f"❌ Error ejecutando consulta: {e}")
            return None
    
    print("✅ Función helper creada:")
    print("   🤖 ask_dividends_expert(): Consulta principal")
    
    print("\n💡 Ejemplo de uso:")
    print('   ask_dividends_expert("¿Qué son los dividendos?")')
    
else:
    print("❌ No se puede crear la función helper: cadena RAG no disponible")
    print("🔄 Asegúrate de haber ejecutado las celdas anteriores correctamente")


### 🧪 Pruebas completas del Sistema RAG

In [ ]:
print("🧪 Probando el sistema RAG completo...")

if 'ask_dividends_expert' in locals():
    test_questions = [
        {
            "question": "¿Qué son los dividendos?",
            "description": "Pregunta básica sobre conceptos fundamentales"
        },
        {
            "question": "¿Cuáles son las principales ventajas de invertir en dividendos?",
            "description": "Pregunta sobre beneficios de la estrategia"
        },
        {
            "question": "¿Cómo puedo evaluar si una empresa paga dividendos sostenibles?",
            "description": "Pregunta práctica sobre análisis"
        },
        {
            "question": "¿Cuáles son los principales ratios financieros para evaluar empresas que reparten dividendos?",
            "description": "Pregunta sobre ratios financieros"
        },
        {
            "question": "¿Qué riesgos debo considerar al invertir en dividendos?",
            "description": "Pregunta sobre gestión de riesgo"
        }
    ]
    
    print(f"🎯 Ejecutando {len(test_questions)} pruebas del sistema RAG...\n")
    
    for i, test in enumerate(test_questions, 1):
        print(f"{'='*80}")
        print(f"🔍 PRUEBA {i}: {test['description']}")
        print(f"❓ PREGUNTA: {test['question']}")
        print(f"{'='*80}")
        
        try:
            result = ask_dividends_expert(
                test['question'], 
                retriever_type="similarity",
                show_sources=True
            )
            
            if result:
                print(f"\n✅ Consulta ejecutada exitosamente")
                print(f"📊 Fuentes utilizadas: {len(result['sources'])}")
            else:
                print(f"\n❌ Error en la consulta")
                
        except Exception as e:
            print(f"\n❌ Error ejecutando prueba {i}: {e}")
        
        print(f"\n{'-'*80}\n")
    
    print(f"\n{'='*80}")
    print("🎉 ¡SISTEMA RAG COMPLETAMENTE FUNCIONAL!")
    print("💡 El asistente de inversión en dividendos está listo para usar")
    print(f"{'='*80}")
    
else:
    print("❌ No se pueden ejecutar pruebas: funciones RAG no disponibles")
    print("🔄 Asegúrate de haber ejecutado todas las celdas anteriores correctamente")


### 🧪 Verificación de que el sistema usa SOLO el RAG (no conocimiento externo)

In [ ]:
print("🧪 Verificando que el sistema usa ÚNICAMENTE el RAG...")

if 'ask_dividends_expert' in locals():
    non_rag_questions = [
        {
            "question": "¿Quién es Lionel Messi?",
            "expected": "No debería saber nada sobre fútbol"
        },
        {
            "question": "¿Cómo funciona la inteligencia artificial?",
            "expected": "No debería tener info sobre IA general"
        },
        {
            "question": "¿Cuál es la capital de Francia?",
            "expected": "No debería saber geografía"
        },
        {
            "question": "¿Qué es Python en programación?",
            "expected": "No debería saber sobre programación"
        },
        {
            "question": "¿Cómo se hace una paella?",
            "expected": "No debería saber cocina"
        }
    ]
    
    print(f"🎯 Ejecutando {len(non_rag_questions)} preguntas FUERA del dominio financiero...\n")
    
    for i, test in enumerate(non_rag_questions, 1):
        print(f"{'='*80}")
        print(f"🔍 PRUEBA ANTI-RAG {i}: {test['expected']}")
        print(f"❓ PREGUNTA: {test['question']}")
        print(f"{'='*80}")
        
        try:
            result = ask_dividends_expert(
                test['question'], 
                retriever_type="similarity",
                show_sources=True
            )
            
            if result:
                print(f"\n✅ Consulta ejecutada exitosamente")
                print(f"📊 Fuentes utilizadas: {len(result['sources'])}")
            else:
                print(f"\n❌ Error en la consulta")
            
            no_info_indicators = [
                "no tengo información",
                "no está en el contexto",
                "no puedo responder",
                "información no disponible",
                "no se encuentra",
                "no hay información"
            ]
            
            has_no_info = any(indicator in result['answer'].lower() for indicator in no_info_indicators)
            
            if has_no_info:
                print(f"✅ **CORRECTO**: El sistema indica que no tiene la información")
            else:
                print(f"⚠️ **ATENCIÓN**: El sistema podría estar usando conocimiento externo")
                print(f"💡 Revisar si el prompt está funcionando correctamente")
            
        except Exception as e:
            print(f"❌ Error ejecutando prueba anti-RAG {i}: {e}")
        
        print(f"\n{'-'*80}\n")
    
    print(f"{'='*80}")
    print("🎯 **RESUMEN DE VERIFICACIÓN:**")
    print("✅ Si todas las respuestas indican 'no tengo información' → RAG funciona correctamente")
    print("⚠️ Si alguna respuesta da información externa → Revisar configuración del prompt")
    print("💡 Un buen sistema RAG debe decir 'no sé' cuando no tiene la información en su base de conocimiento")
    print(f"{'='*80}")
    
else:
    print("❌ No se puede ejecutar la verificación: función RAG no disponible")
    print("🔄 Asegúrate de haber ejecutado todas las celdas anteriores correctamente")